Explore the raw velocity data

In [2]:
import numpy as np

In [7]:
# code copied from William
Re = 600

vfield = np.load(
    f"../hw2/resources/von_karman_street/vortex_street_velocities_Re_{Re}_largefile.npz",
    allow_pickle=True
)
print("Velocity field data as shape: {}".format(vfield.shape))

# Calculate the vortivity, which is the curl of the velocity field
vort_field = np.diff(vfield, axis=1)[..., :1, 1] + np.diff(vfield, axis=2)[:, :-1, :, 0]

Velocity field data as shape: (3000, 256, 128, 2)


Implement principle component analysis

In [ ]:
from sklean.base import BaseEstimator, TransformerMixin